In [1]:
from tensorflow.keras.models import load_model
import librosa
import pygame
import numpy as np
import time
import json
from uuid import uuid4
import requests

class Command:
    def __init__(self, commandSentense):
        self.id = str(uuid4())
        self.commandSentense = commandSentense

threshold = 0.6
command_labels = ['bongeszo', 'fazok', 'homerseklet_alap','homerseklet_vissza','klima_be','klima_le','melegem_van','mennyi_az_ido','vicc','villany_fel','villany_le' ]
model = load_model('./convloss05catacc98.h5')
pygame.mixer.init()
sound = pygame.mixer.Sound("yes.wav")
def preprocess_log_mel_spectrogram(file_path,n_mels=128,  duration=4.0):
    # Load the audio file
    
    waveform, sr = librosa.load(file_path, duration=duration)    
    print(sr)
    hop_length = int(sr * (0.02 - 0.01))
    # Extract the Log-Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(waveform, sr=sr, n_mels=n_mels, hop_length=hop_length)
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

    return log_mel_spectrogram

pygame 2.4.0 (SDL 2.26.4, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
spec_to_predict = preprocess_log_mel_spectrogram('./test_voices/bongeszo.wav')
spec_to_predict.shape

# A modell elvárt input shape-je
desired_shape = model.input_shape

# Kiszámítjuk a szükséges padding-et az első két dimenzióra
pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

# Padding-et alkalmazunk a spektrogramra
padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')

# Az eredmény shape-je már megfelel a modell elvárt input shape-jének
print(padded_spec.shape)


spec_to_predict = np.expand_dims(padded_spec, axis=-1)
print(spec_to_predict.shape)


spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)
print(spec_to_predict_batch.shape)

np.set_printoptions(suppress=True)

predictions = model.predict(spec_to_predict_batch)

print(predictions)


predicted_class_index = np.argmax(predictions)

predicted_class = command_labels[predicted_class_index]
print(predicted_class)

In [2]:
import pyaudio
import numpy as np

def record_audio(duration, sample_rate):
    chunk_size = 1024
    audio_buffer = []

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=chunk_size)

    num_frames = int(duration * sample_rate / chunk_size)
    for _ in range(num_frames):
        data = stream.read(chunk_size)
        audio_buffer.append(np.frombuffer(data, dtype=np.int16))

    stream.stop_stream()
    stream.close()
    p.terminate()

    audio_data = np.concatenate(audio_buffer)
    return audio_data

# Hangrögzítés 4 másodpercig, 44100 mintavételezési ráta mellett
duration = 4  # Másodpercekben
sample_rate = 22050  # Mintavételezési ráta


In [3]:
import librosa
import librosa.util
import numpy as np

def preprocess_log_mel_spectrogram_pyaudio(audio_data, sr, n_mels=128, duration=2.0):
    # Convert audio data to floating-point
    audio_data = librosa.util.buf_to_float(audio_data)

    hop_length = int(sr * (0.02 - 0.01))
    # Extract the Log-Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(audio_data, sr=sr, n_mels=n_mels, hop_length=hop_length)
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

    return log_mel_spectrogram


In [39]:
audio_data = record_audio(duration, sample_rate)

In [ ]:
spec_to_predict = preprocess_log_mel_spectrogram_pyaudio(audio_data, 22050)

In [ ]:
spec_to_predict.shape

In [20]:
pred_array = []

In [15]:
def command_prediction_to_array():
    audio_data = record_audio(2.0, sample_rate)
    spec_to_predict = preprocess_log_mel_spectrogram_pyaudio(audio_data, 22050)
    spec_to_predict.shape
    desired_shape = model.input_shape

# Kiszámítjuk a szükséges padding-et az első két dimenzióra
    pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

# Padding-et alkalmazunk a spektrogramra
    padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')

# Az eredmény shape-je már megfelel a modell elvárt input shape-jének
#print(padded_spec.shape)


    spec_to_predict = np.expand_dims(padded_spec, axis=-1)
#print(spec_to_predict.shape)


    spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)
    
    pred_array.append(spec_to_predict_batch)

In [ ]:
audio_data = record_audio(2.0, sample_rate)
spec_to_predict = preprocess_log_mel_spectrogram_pyaudio(audio_data, 22050)
spec_to_predict.shape
desired_shape = model.input_shape

# Kiszámítjuk a szükséges padding-et az első két dimenzióra
pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

# Padding-et alkalmazunk a spektrogramra
padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')

# Az eredmény shape-je már megfelel a modell elvárt input shape-jének
#print(padded_spec.shape)


spec_to_predict = np.expand_dims(padded_spec, axis=-1)
#print(spec_to_predict.shape)


spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)
#print(spec_to_predict_batch.shape)

np.set_printoptions(suppress=True)

predictions = model.predict(spec_to_predict_batch)
print(predictions)

predicted_class_index = np.argmax(predictions)

predicted_class = command_labels[predicted_class_index]
print(predicted_class)

In [4]:
# sound.play()
# pygame.time.wait(int(sound.get_length() * 1000))

# time.sleep(1)
# print("mewhet")
def command_prediction():
    audio_data = record_audio(2.0, sample_rate)
    spec_to_predict = preprocess_log_mel_spectrogram_pyaudio(audio_data, 22050)
    spec_to_predict.shape
    desired_shape = model.input_shape

# Kiszámítjuk a szükséges padding-et az első két dimenzióra
    pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

# Padding-et alkalmazunk a spektrogramra
    padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')

# Az eredmény shape-je már megfelel a modell elvárt input shape-jének
#print(padded_spec.shape)


    spec_to_predict = np.expand_dims(padded_spec, axis=-1)
#print(spec_to_predict.shape)


    spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)
#print(spec_to_predict_batch.shape)

    np.set_printoptions(suppress=True)

    predictions = model.predict(spec_to_predict_batch)

    print(predictions)

    if np.max(predictions) > threshold:
        predicted_class_index = np.argmax(predictions)
        predicted_class = command_labels[predicted_class_index]
        print(predicted_class)
    else:
        return "nincs_match"

    ##predicted_class_index = np.argmax(predictions)

    ##predicted_class = command_labels[predicted_class_index]
    print(predicted_class)
    return predicted_class

In [5]:
import threading
import time



In [21]:
# for i in range(8):    
#     szal = threading.Thread(target=command_prediction_to_array)
#     szal.start()
#     time.sleep(0.25)

C:\Users\Laci\AppData\Local\Temp\ipykernel_11948\1419762101.py:11: FutureWarning: Pass y=[ 0.          0.00003052 -0.00003052 ...  0.         -0.00003052
 -0.00006104] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_spectrogram = librosa.feature.melspectrogram(audio_data, sr=sr, n_mels=n_mels, hop_length=hop_length)
C:\Users\Laci\AppData\Local\Temp\ipykernel_11948\1419762101.py:11: FutureWarning: Pass y=[ 0.00021362  0.00036621  0.00027466 ... -0.00021362 -0.00018311
 -0.00015259] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_spectrogram = librosa.feature.melspectrogram(audio_data, sr=sr, n_mels=n_mels, hop_length=hop_length)
C:\Users\Laci\AppData\Local\Temp\ipykernel_11948\1419762101.py:11: FutureWarning: Pass y=[-0.00045776 -0.00024414  0.00030518 ...  0.          0.
 -0.00003052] as keyword args. From version 0.10 passing these as positional arguments will result in an err

In [18]:
# pred_array

[array([[[[-39.42319 ],
          [-38.749382],
          [-41.33654 ],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]],
 
         [[-52.694977],
          [-53.818394],
          [-59.06141 ],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]],
 
         [[-60.88799 ],
          [-60.03288 ],
          [-61.655197],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]],
 
         ...,
 
         [[-80.      ],
          [-80.      ],
          [-80.      ],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]],
 
         [[-80.      ],
          [-80.      ],
          [-80.      ],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]],
 
         [[-80.      ],
          [-80.      ],
          [-80.      ],
          ...,
          [  0.      ],
          [  0.      ],
          [  0.      ]]]], dtype=float3

In [22]:
# command_arry = []
# np.set_printoptions(suppress=True)
# for specgrams in pred_array:

#     predictions = model.predict(specgrams)

#     print(predictions)

#     if np.max(predictions) > threshold:
#         predicted_class_index = np.argmax(predictions)
#         predicted_class = command_labels[predicted_class_index]
#         command_arry.append(predicted_class)
#         print(predicted_class)
#     else:
#          command_arry.append("nincs_match")

#     ##predicted_class_index = np.argmax(predictions)

#     ##predicted_class = command_labels[predicted_class_index]
#     # print(predicted_class)
#     # return predicted_class

1/1 [==============================] - 0s 15ms/step
[[0.00000268 0.         0.0000065  0.         0.         0.00000067
  0.         0.99680054 0.00315252 0.         0.00003701]]
mennyi_az_ido
1/1 [==============================] - 0s 19ms/step
[[0.00042152 0.         0.0028889  0.         0.         0.
  0.         0.00000021 0.9966807  0.         0.00000864]]
vicc
1/1 [==============================] - 0s 16ms/step
[[0.43207952 0.         0.56368625 0.         0.         0.00000015
  0.         0.00000003 0.00413762 0.         0.00009641]]
1/1 [==============================] - 0s 17ms/step
[[0.8844772  0.         0.00016074 0.         0.         0.00000015
  0.         0.00000001 0.1153606  0.         0.00000136]]
bongeszo
1/1 [==============================] - 0s 17ms/step
[[0.00108724 0.         0.00000001 0.00000009 0.         0.00000005
  0.00023307 0.00000422 0.9986753  0.         0.00000002]]
vicc
1/1 [==============================] - 0s 16ms/step
[[0.00000287 0.00000002 0.  

In [8]:
payload = Command(command_prediction())
#payload = Command("sajt")
url = "http://localhost:5025/Command"
headers = {'Content-Type': 'application/json'}
data = json.dumps(payload.__dict__)
response = requests.post(url, headers=headers, data=data)

print(response.text)

yoo


In [82]:
# sound.play()
# pygame.time.wait(int(sound.get_length() * 1000))

2005